In [1]:
# from typing import Any, List, Union, Dict
# from pydantic import BaseModel, field_validator, ValidationError, PrivateAttr


import obi2


nested_multi = obi2.Stimulus(nested_param1=[1.5, 2.5], nested_param2=[3.5, 7.0])
simulation_campaign_template = obi2.SimulationCampaignTemplate(nested={"config": nested_multi})
campaign = obi2.Campaign(template_instance=simulation_campaign_template)
print(campaign.coord_instances)

# print(simulation_campaign_template.multi_params)
# print(simulation_campaign_template.generate_grid_scan_coords())












[Simulation(nested={'config': Stimulus(nested_param1=1.5, nested_param2=3.5)}), Simulation(nested={'config': Stimulus(nested_param1=1.5, nested_param2=7.0)}), Simulation(nested={'config': Stimulus(nested_param1=2.5, nested_param2=3.5)}), Simulation(nested={'config': Stimulus(nested_param1=2.5, nested_param2=7.0)})]


In [ ]:
# # Valid Simulation instance (SingleType) with direct nested object
# nested_single = Stimulus(nested_param1=1.5, nested_param2=2.5)  # ✅ Must be single floats
# print(nested_single)
# simulation = Simulation(param1=2.0, param2=3.0, nested={"config": nested_single})
# print(simulation)

# Valid SimulationCampaignTemplate instance (MultiType) with direct nested object

# # Valid Simulation with nested dictionary containing BaseModel
# simulation_with_dict = Simulation(
#     param1=2.0, 
#     param2=3.0, 
#     nested={"config": Stimulus(nested_param1=2.2, nested_param2=4.4)}
# )
# print(simulation_with_dict)

# # Valid SimulationCampaignTemplate with nested dictionary containing BaseModel
# simulation_campaign_template_with_dict = SimulationCampaignTemplate(
#     param1=[2.0, 4.0], 
#     param2=3.0, 
#     nested={"config": Stimulus(nested_param1=[1.1, 2.2], nested_param2=3.3)}
# )
# print(simulation_campaign_template_with_dict)

# print("\nNew Test\n")



# ❌ Nested dictionary contains a BaseModel with a list (not allowed in Simulation)
try:
    invalid_simulation_dict = Simulation(
        param1=2.0, 
        param2=3.0, 
        nested={"config": Stimulus(nested_param1=[1.5, 2.5], nested_param2=3.5)}
    )
except ValueError as e:
    print(e)

# ❌ Nested dictionary contains a BaseModel with a string (not allowed in SimulationCampaignTemplate)
try:
    invalid_simulation_campaign_template_dict = SimulationCampaignTemplate(
        param1=[2.0, 4.0], 
        param2=3.0, 
        nested={"config": Stimulus(nested_param1="invalid", nested_param2=3.5)}
    )
except ValueError as e:
    print(e)